# Analyze simulation results based on country-wide network
* underlying network: entire country = Austria
* analyze effects (lost patients, average displacement and free capacity) in single states and on country level
* patients steps restricted to max-dist from starting doctor
* SIM tries 'max_dist_trials' times to find a doc within reach, then it just chooses another one

In [ ]:
import pandas as pd
import numpy as np
import os
from os.path import join

In [ ]:
def analyze_results(iterations,shocks,remov,alpha,max_steps,threshold,kd,max_dist,max_dist_trials,min_pats):
    '''   
    Set parameters and load datafiles with doc info 
    * normalize lost patients in states
    * calculate all unique patients, unique docs per speciality..
    * define parameters
    '''
    
    #list of all specialities 'IM','KI','PSY','ORTR','RAD','DER','URO','HNO','CH','NEU','AU','GGH','AM'
    doctors = list(['IM','KI','PSY','ORTR','RAD','DER','URO','HNO','CH','NEU','AU','GGH','AM'])
    states = {'state':['Burgenland','Kärnten','Niederösterreich','Oberösterreich','Salzburg','Steiermark',
                       'Tirol','Vorarlberg','Wien'],
             'state_id':[1,2,3,4,5,6,7,8,9]}
    network = 'Österreich' # what is the underlying network for doc connections 


    # information selection criteria
    patient_type = 'total'
    capacity_type = 'hour-based'
    timeframe = 'quarterly'


    ### dataframe with total patient numbers and unique doc numbers per state and per specialty
    N = pd.DataFrame.from_dict(states)
    N.set_index('state',inplace=True)

    for specialization in doctors:
        N[specialization+'_total'] = 0
        N[specialization+'_cap_total'] = 0
        N[specialization+'_unique_docs'] = 0
        for bez in N.index:
            dinfo = pd.read_csv('data/doctor_info_bez='+network+'_spec='+specialization+'_ptype='+patient_type+
                                '_ctype='+capacity_type+'_tf='+timeframe+'_th='+str(threshold)+'.csv',
                                usecols=list(['number_of_patients','capacity','gemeinde']))
            dinfo.gemeinde = dinfo.gemeinde.astype(str)
            N.loc[bez,specialization+'_total'] = dinfo[dinfo.gemeinde.str.startswith(str(N.loc[bez,'state_id']))].number_of_patients.sum()
            N.loc[bez,specialization+'_cap_total'] = dinfo[dinfo.gemeinde.str.startswith(str(N.loc[bez,'state_id']))].capacity.sum()
            N.loc[bez,specialization+'_unique_docs'] = len(dinfo[dinfo.gemeinde.str.startswith(str(N.loc[bez,'state_id']))])

    N.to_excel('results/states_doc_info_{}_{}_{}.xlsx'.format(patient_type, capacity_type, timeframe))

    '''
    Check results for single states based on entire network
    * save all simulation data in one dataframe for seaborn plots
    * sum up all lost patients from previous shocks (only possible on country-level, missing info on patients residence)
    '''

    dta = pd.DataFrame(columns=['run','shock','avg_displacement','N_lost_patients','N_lost_patients_summed',
                                'free_capacity_country','free_capacity_state','disconnected_capacity',
                                'state','specialty','lost_patients_country','incorrect_displacements',
                                'N_lost_patients_state_summed'])

    for doc in doctors:  
        ### read in the data file
        sim_params = 'patient_dynamics_iter{}_shocks{}_remove{}_alpha{}_maxs{}_th{}_kd{}_maxdist{}_maxdisttrials{}_minpats{}_{}_Final_.csv'\
                    .format(iterations, shocks, remov, alpha, max_steps, threshold, kd,max_dist,max_dist_trials,min_pats,doc)
        dta_load = pd.read_csv('results/'+sim_params)
        for bez in N.state_id:
            dta2 = dta_load[dta_load.state==bez].copy()
            dta2['state'] = N[N.state_id == bez].index.item()
            dta2['specialty'] = doc
            dta2['lost_patients_country'] = dta2.N_lost_patients_summed/N.loc[:,doc+'_total'].sum()*100
            dta2['lost_patients_state'] = dta2.N_lost_patients_state_summed/N.loc[N.state_id==bez,doc+'_total'].sum()*100
            dta2['free_capacity_country'] = dta2.free_capacity_country/N.loc[:,doc+'_cap_total'].sum()*100
            dta2['free_capacity_state'] = dta2.free_capacity_state/N.loc[N.state_id==bez,
                                                                                 doc+'_cap_total'].item()*100
            dta = pd.concat([dta,dta2])


    dta.reset_index(drop=True,inplace=True)


    dta['num_unique_docs'] = 0
    for doc in doctors:
        dta.loc[(dta.specialty == doc),'num_unique_docs'] = N.loc[:,doc+'_unique_docs'].sum() 

    dta['perc_docs_removed'] = dta.shock / dta.num_unique_docs * 100
        
    
    ### calculate remaining free capacity filling up
    dta['remaining_FC_filled'] = 0
    
    for doc in dta.specialty.unique():  
        for bez in dta.state.unique():
            maxv = dta.loc[(dta.state==bez)&(dta.specialty==doc),'free_capacity_state'].max()
            dta.loc[(dta.state==bez)&(dta.specialty==doc),'remaining_FC_filled'] = \
                        dta.loc[(dta.state==bez)&(dta.specialty==doc),'free_capacity_state'].values * (100/maxv)

    dta.remaining_FC_filled = 100-dta.remaining_FC_filled
    return dta

## Set parameters of SIM to analyze

In [ ]:
iterations = 100     # number of sim iterations
shocks = 5000        # number of maximum doctors to remove
remov = 1            # number of docs removed in each step
alpha = 0.0          # teleportation probability
max_steps = 10       # max steps for patients before becoming lost
kd = 1               # keep disconnected doctors 0/1
threshold = 0.9      # capacity threshold for calculation
max_dist = 100       # maximum travelling distance
max_dist_trials = 10 # sim trials to find doc within max distance before using some other doc  
min_pats = 2         # minimum number of patients for valid connection in adj.matrix

In [ ]:
### run function for selected setting
res = analyze_results(iterations,shocks,remov,alpha,max_steps,threshold,kd,max_dist,
                    max_dist_trials,min_pats)

## Results for table in paper
* for every state & specialist when limit for LP and FC are reached

In [ ]:
res_copy = res.copy()

In [ ]:
res_copy.to_csv('results/DF_results_Final.csv',index=False)

In [ ]:
res_grouped = res_copy.groupby(['state','specialty','shock']).mean()[['avg_displacement',
    'lost_patients_state','remaining_FC_filled','perc_docs_removed','num_unique_docs']].round(2).reset_index()
res_grouped.to_csv('results/DF_results_mean_Final.csv',index=False)

## FC and LP limits

In [ ]:
### select df up to limit
LPlimit = 1
FClimit = 20

LP = res_grouped[res_grouped.lost_patients_state<=LPlimit]
FC = res_grouped[res_grouped.remaining_FC_filled<=FClimit]

In [ ]:
### lost patients per state and specialist for selected limit
idx = LP.groupby(['state','specialty'])['lost_patients_state'].transform(max) == LP['lost_patients_state']
LP = LP[idx]
LP = LP.sort_values(['state','specialty','perc_docs_removed'],ascending=True).drop_duplicates(subset=
        ['state','specialty'], keep='last')[['state','specialty','shock','lost_patients_state',
                                             'perc_docs_removed']]

LP.reset_index(drop=True,inplace=True)

In [ ]:
### free capacity per state and specialist for selected limit
idx = FC.groupby(['state','specialty'])['remaining_FC_filled'].transform(max) == FC['remaining_FC_filled']
FC = FC[idx]
FC = FC.sort_values(['state','specialty','perc_docs_removed'],ascending=True).drop_duplicates(subset=
        ['state','specialty'], keep='last')[['state','specialty','shock','remaining_FC_filled',
                                             'perc_docs_removed']]

FC.reset_index(drop=True,inplace=True)

In [ ]:
### add info on state ID
states = {'state':['Burgenland','Kärnten','Niederösterreich','Oberösterreich',
                    'Salzburg','Steiermark','Tirol','Vorarlberg','Wien'],'state_id':[1,2,3,4,5,6,7,8,9]}

LP['state_ID'] = 0
FC['state_ID'] = 0
for s,i in zip(states['state'],states['state_id']):
    LP.loc[LP.state==s,'state_ID'] = i
    FC.loc[FC.state==s,'state_ID'] = i

In [ ]:
### rename
FC.rename(columns={'remaining_FC_filled':'perc_free_capacity','shock':'num_docs_removed'},inplace=True)
LP.rename(columns={'lost_patients_state':'perc_lost_patients','shock':'num_docs_removed'},inplace=True)

### change dtype
LP.num_docs_removed = LP.num_docs_removed.astype(int)
FC.num_docs_removed = FC.num_docs_removed.astype(int)

### switch order
LP = LP[['state', 'state_ID', 'specialty', 'num_docs_removed', 'perc_lost_patients','perc_docs_removed']]
FC = FC[['state', 'state_ID', 'specialty', 'num_docs_removed', 'perc_free_capacity','perc_docs_removed']]

In [ ]:
FC.to_csv('results/FC_state_specialty_FClimit{}.csv'.format(FClimit),index=False)
LP.to_csv('results/LP_state_specialty_LPlimit{}.csv'.format(LPlimit),index=False)

In [ ]:
FC.rename(columns={'num_docs_removed':'fc_num_docs_removed',
                   'perc_free_capacity':'fc_perc_free_capacity',
                   'perc_docs_removed':'fc_perc_docs_removed'},inplace=True)

LP.rename(columns={'num_docs_removed':'lp_num_docs_removed',
                   'perc_lost_patients':'lp_perc_lost_patients',
                   'perc_docs_removed':'lp_perc_docs_removed'},inplace=True)

FC['lp_num_docs_removed'] = LP['lp_num_docs_removed'] 
FC['lp_perc_lost_patients'] = LP['lp_perc_lost_patients'] 
FC['lp_perc_docs_removed'] = LP['lp_perc_docs_removed'] 

In [ ]:
FC.to_csv('results/FCLP_limits_{}_{}.csv'.format(FClimit,LPlimit),index=False)